In [1]:
import torch

# 反向传播
即误差反向传播，是训练神经网络的重要方法。

参照tensor.ipynb所写的一个简单的例子

In [19]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1) * 10
y = 2 * x + (5 + torch.randn(20, 1))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度

# 设置学习率lr为0.1
lr = 0.1

for iteration in range(100):
    # 前向传播
    wx = torch.mul(w, x) #将两张量中元素
    y_pred = torch.add(wx, b)

    
    # 计算loss
    loss = (0.5 * (y-y_pred)**2).mean()
    print("*"*60)
    print(f"第{iteration}次反向传播前的梯度为:")
    try:
        print(w.grad.data,b.grad.data)
    except :
        print("梯度不存在")
    # 反向传播
    loss.backward()
    print(f"第{iteration}次反向传播后的梯度为:")
    print(w.grad.data,b.grad.data)
    print("*"*60)
    # 更新参数
    b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
    w.data.sub_(lr * w.grad)

    # 梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()

print(w.data, b.data)

************************************************************
第0次反向传播前的梯度为:
梯度不存在
第0次反向传播后的梯度为:
tensor([-144.5525]) tensor([-23.0967])
************************************************************
************************************************************
第1次反向传播前的梯度为:
tensor([0.]) tensor([0.])
第1次反向传播后的梯度为:
tensor([353.6027]) tensor([53.8844])
************************************************************
************************************************************
第2次反向传播前的梯度为:
tensor([0.]) tensor([0.])
第2次反向传播后的梯度为:
tensor([-863.6285]) tensor([-134.1645])
************************************************************
************************************************************
第3次反向传播前的梯度为:
tensor([0.]) tensor([0.])
第3次反向传播后的梯度为:
tensor([2110.6216]) tensor([325.3762])
************************************************************
************************************************************
第4次反向传播前的梯度为:
tensor([0.]) tensor([0.])
第4次反向传播后的梯度为:
tensor([-5156.8525]) tensor([-797.4445])
*****

可以看到，参数在经过反向传播后获得了梯度。
但是有个问题，torch是怎么知道该计算谁的梯度呢


通过查询资料可以得知，torch.backward()默认情况下只累积叶子节点张量的梯度

In [16]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1) * 10
y = 2 * x + (5 + torch.randn(20, 1))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度
wx = torch.mul(w, x) #将两张量中元素
y_pred = torch.add(wx, b)

print("x is leaf:",x.is_leaf)
print("y is leaf:",y.is_leaf)
print("w is leaf:",w.is_leaf)
print("b is leaf:",b.is_leaf)
print("wx is leaf:",wx.is_leaf)
print("y_pred is leaf:",y_pred.is_leaf)

x is leaf: True
y is leaf: True
w is leaf: True
b is leaf: True
wx is leaf: False
y_pred is leaf: False


那是不是意味着x,y同样也获得了梯度呢

In [23]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1) * 10
y = 2 * x + (5 + torch.randn(20, 1))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度

# 设置学习率lr为0.1
lr = 0.1

for iteration in range(100):
    # 前向传播
    wx = torch.mul(w, x) #将两张量中元素
    y_pred = torch.add(wx, b)

    
    # 计算loss
    loss = (0.5 * (y-y_pred)**2).mean()
    print("*"*60)
    print(f"第{iteration}次反向传播前的梯度为:")
    try:
        print(w.grad.data,b.grad.data,x.grad.data,y.grad.data)
    except :
        print("梯度不存在")
    # 反向传播
    loss.backward()
    print(f"第{iteration}次反向传播后的梯度为:")
    print(w.grad.data,b.grad.data)
    print(x.grad.data,y.grad.data)
    print("*"*60)
    # 更新参数
    b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
    w.data.sub_(lr * w.grad)

    # 梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()

print(w.data, b.data)

************************************************************
第0次反向传播前的梯度为:
梯度不存在
第0次反向传播后的梯度为:
tensor([-193.3717]) tensor([-28.7433])


d:\conda\envs\d2l\lib\site-packages\torch\_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:475.)
  return self._grad


AttributeError: 'NoneType' object has no attribute 'data'

显然不太行，发现w,b后面有requires_grad，那如果去掉是不是连w,b也没法求梯度了呢？

In [26]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1) * 10
y = 2 * x + (5 + torch.randn(20, 1))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=False)
b = torch.zeros((1), requires_grad=False)   # 这俩都需要求梯度

# 设置学习率lr为0.1
lr = 0.1

for iteration in range(100):
    # 前向传播
    wx = torch.mul(w, x) #将两张量中元素
    y_pred = torch.add(wx, b)

    
    # 计算loss
    loss = (0.5 * (y-y_pred)**2).mean()
    print("*"*60)
    print(f"第{iteration}次反向传播前的梯度为:")
    try:
        print(w.grad.data,b.grad.data,x.grad.data,y.grad.data)
    except :
        print("梯度不存在")
    # 反向传播
    loss.backward()
    print(f"第{iteration}次反向传播后的梯度为:")
    print(w.grad.data,b.grad.data)
    print(x.grad.data,y.grad.data)
    print("*"*60)
    # 更新参数
    b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
    w.data.sub_(lr * w.grad)

    # 梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()

print(w.data, b.data)

************************************************************
第0次反向传播前的梯度为:
梯度不存在


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

确实，那么如果我们为x,y也加入requires_grad=True ，那是不是意味着x,y同样也能获取梯度呢

In [29]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1, requires_grad=True)
y = 2 * x + (5 + torch.randn(20, 1, requires_grad=True))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度

# 设置学习率lr为0.1
lr = 0.1

for iteration in range(100):
    # 前向传播
    wx = torch.mul(w, x) #将两张量中元素
    y_pred = torch.add(wx, b)

    
    # 计算loss
    loss = (0.5 * (y-y_pred)**2).mean()
    print("*"*60)
    print(f"第{iteration}次反向传播前的梯度为:")
    try:
        print(w.grad.data,b.grad.data)
    except :
        print("梯度不存在")
    # 反向传播
    loss.backward()
    print(f"第{iteration}次反向传播后的梯度为:")
    print(w.grad.data,b.grad.data)
    print("*"*60)
    # 更新参数
    b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
    w.data.sub_(lr * w.grad)

    # 梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()

print(w.data, b.data)

************************************************************
第0次反向传播前的梯度为:
梯度不存在
第0次反向传播后的梯度为:
tensor([-3.5335]) tensor([-6.6732])
************************************************************
************************************************************
第1次反向传播前的梯度为:
tensor([0.]) tensor([0.])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

出现了些奇怪的错误，报错的意思就是，因为某个带有梯度信息的变量在被执行了一次后，这些梯度信息就被计算图释放掉了，而我们的代码却尝试第二次反向传播的时候来访问这些变量(梯度信息)。

在查询资料后发现在backward后添加retain_graph=True就可以在每次反向传播后暂时保留中间节点的梯度值，

In [42]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1, requires_grad=True)

y = 2 * x + (5 + torch.randn(20, 1, requires_grad=True))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度

# 设置学习率lr为0.1
lr = 0.1

for iteration in range(100):
    # 前向传播
    wx = torch.mul(w, x) #将两张量中元素
    y_pred = torch.add(wx, b)

    
    # 计算loss
    loss = (0.5 * (y-y_pred)**2).mean()

    print("*"*60)
    print(f"第{iteration}次反向传播前的梯度为:")
    try:
        print(w.grad.data,b.grad.data,x.grad.data,y.grad.data)
    except :
        print("梯度不存在")
    # 反向传播
    loss.backward(retain_graph=True)
    print(f"第{iteration}次反向传播后的梯度为:")
    t = x.grad.data if x.grad != None  else None
    t1 = y.grad.data if y.grad != None else None
    print(w.grad.data,b.grad.data,t,t1)
    print("*"*60)
    # 更新参数
    b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
    w.data.sub_(lr * w.grad)

    # 梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()

print(w.data, b.data)

************************************************************
第0次反向传播前的梯度为:
梯度不存在
第0次反向传播后的梯度为:
tensor([-2.2859]) tensor([-5.9602]) tensor([[0.3021],
        [0.3650],
        [0.3853],
        [0.3502],
        [0.4887],
        [0.4344],
        [0.4268],
        [0.4804],
        [0.4478],
        [0.4632],
        [0.3373],
        [0.3467],
        [0.4078],
        [0.4276],
        [0.3957],
        [0.2453],
        [0.3994],
        [0.4038],
        [0.4239],
        [0.2832]]) None
************************************************************
************************************************************
第1次反向传播前的梯度为:
梯度不存在
第1次反向传播后的梯度为:
tensor([-2.0005]) tensor([-5.2769]) tensor([[0.5181],
        [0.6329],
        [0.6611],
        [0.6051],
        [0.8593],
        [0.7505],
        [0.7449],
        [0.8326],
        [0.7834],
        [0.8032],
        [0.5752],
        [0.5979],
        [0.7056],
        [0.7420],
        [0.6898],
        [0.4048],
        [0.6960],
     

未经过任何计算的x有了梯度，而经过计算的y没有，那么会是这个原因吗

In [45]:
# 首先我们得有训练样本X，Y， 这里我们随机生成
x = torch.rand(20, 1, requires_grad=True) +6

y = 2 * x + (5 + torch.randn(20, 1, requires_grad=True))

# 构建线性回归函数的参数
w = torch.randn((1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)   # 这俩都需要求梯度

# 设置学习率lr为0.1
lr = 0.1

for iteration in range(100):
    # 前向传播
    wx = torch.mul(w, x) #将两张量中元素
    y_pred = torch.add(wx, b)

    
    # 计算loss
    loss = (0.5 * (y-y_pred)**2).mean()

    print("*"*60)
    print(f"第{iteration}次反向传播前的梯度为:")
    try:
        print(w.grad.data,b.grad.data,x.grad.data,y.grad.data)
    except :
        print("梯度不存在")
    # 反向传播
    loss.backward(retain_graph=True)
    print(f"第{iteration}次反向传播后的梯度为:")
    t = x.grad.data if x.grad != None  else None
    t1 = y.grad.data if y.grad != None else None
    print(w.grad.data,b.grad.data,t,t1)
    print("*"*60)
    # 更新参数
    b.data.sub_(lr * b.grad)    # 这种_的加法操作时从自身减，相当于-=
    w.data.sub_(lr * w.grad)

    # 梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()

print(w.data, b.data)

************************************************************
第0次反向传播前的梯度为:
梯度不存在
第0次反向传播后的梯度为:
tensor([-2.8534]) tensor([-5.7790]) tensor([[-0.1618],
        [-0.1793],
        [-0.1260],
        [-0.1605],
        [-0.1202],
        [-0.1990],
        [-0.1493],
        [-0.1229],
        [-0.2035],
        [-0.1558],
        [-0.1348],
        [-0.1343],
        [-0.1691],
        [-0.2098],
        [-0.1676],
        [-0.1083],
        [-0.1101],
        [-0.1402],
        [-0.1767],
        [-0.1312]]) None
************************************************************
************************************************************
第1次反向传播前的梯度为:
梯度不存在
第1次反向传播后的梯度为:
tensor([-2.4753]) tensor([-5.0599]) tensor([[-0.3801],
        [-0.4272],
        [-0.2868],
        [-0.3795],
        [-0.2745],
        [-0.4802],
        [-0.3480],
        [-0.2794],
        [-0.4870],
        [-0.3659],
        [-0.3177],
        [-0.3078],
        [-0.3957],
        [-0.4994],
        [-0.4012],
      

给x加点东西，发现真的没法计算梯度了。

那么如何计算参数梯度的前提要求就已经确定了
1. 使用torch定义一个未经过运算的张量
2. 为张量设定requires_grad=True
3. 确保张量为叶节点

实操部分结束，那么该回到枯燥的理论部分了
<p id = "end"><p>

[返回深度学习笔记](../../深度学习.md)

[直接前往反向传播部分](./backward.ipynb)